In [1]:
#############THIS CODE IS WRITTEN FOR PAPER IRMRIS###########
############# FOR ABLATION STUDY EVALUATE##################
############Dated: 20-02-2022######################

In [2]:
##################HERE WE USED WITH SKIP CONNECTIONS#############

In [3]:
from numpy import clip
from keras.layers.advanced_activations import PReLU
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import Conv2D, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.nn import depth_to_space
from tensorflow import clip_by_value
import HelpFunc as hf
from tensorflow.keras.layers import Input, Conv2D, Activation, SeparableConv2D
from tensorflow.keras.layers import PReLU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Concatenate, add, Activation, Dropout

In [4]:
def ResNet_block():
    _input = Input(shape=(None, None, 1))

    C1 = Conv2D(1, (3, 3), strides=(1, 1), padding='same')(_input)
    PR1 = PReLU(shared_axes=[1,2])(C1)
    
    C2 = Conv2D(1, (1,1), strides=(1, 1), padding='same')(PR1)

    sum1 = add(inputs=[_input ,C1,C2])

    
    C3 = Conv2D(1,(3,3), strides=(1, 1), padding='same')(sum1)
    PR2 = PReLU(shared_axes=[1,2])(C3)
  

    C4 = Conv2D(1, (1,1), strides=(1, 1), padding='same')(PR2)

    sum2 = add(inputs=[_input, sum1, C4])
 
    
    model = Model(inputs=_input, outputs=sum2)

    return model
model = ResNet_block()

In [5]:
def Multi_Path_Inception_Block():
    _input = Input(shape=(None, None, 1))
    C1 = Conv2D(1, (1,1),strides=(1,1),padding='same')(_input)

    
    #First barnch Second Block
    C2 = Conv2D(22, (3,3),strides=(1,1),padding='same')(C1)
    PR2 = PReLU(shared_axes=[1,2])(C2)

    C3 = Conv2D(22, (3,3),strides=(1,1),padding='same')(PR2)
    PR3 = PReLU(shared_axes=[1,2])(C3)
    

    
    #Second Branch
    C4 = Conv2D(22, (5,5),strides=(1,1),padding='same')(C1)
    PR4 = PReLU(shared_axes=[1,2])(C4)


    C5 = Conv2D(22, (5,5),strides=(1,1),padding='same')(PR4)
    PR5 = PReLU(shared_axes=[1,2])(C5)
   
    #Third Branch
    C6 = Conv2D(22, (7,7),strides=(1,1),padding='same')(C1)
    PR6 = PReLU(shared_axes=[1,2])(C6)
    

    C7 = Conv2D(22, (7,7),strides=(1,1),padding='same')(PR6)
    PR7 = PReLU(shared_axes=[1,2])(C7)
    

    
    sum3 = add(inputs=[C1, PR3, PR5, PR7])
    
  
    C8 = Conv2D(1, (1,1),strides=(1,1),padding='same')(sum3)
    
    model = Model(inputs=_input, outputs=C8)

    return model
model = Multi_Path_Inception_Block()


In [6]:
def IRMRIS():
    scale = 4
    X_in = Input(shape=(None, None, 1))

    X1 = Conv2D(64, (5,5), padding='same', activation='relu')(X_in)
    X2 = Conv2D(1, (3,3), padding='same', activation='relu')(X1)
    X3 = ResNet_block()(X2)
    sum1 = add(inputs=[X2,X3])
    X4 = ResNet_block()(sum1)
    sum2 = add(inputs=[sum1, X4])
    X5 = ResNet_block()(sum2)
    sum3 = add(inputs=[sum2,X5])
    X6 = ResNet_block()(sum3)
    sum4 = add(inputs=[sum3,X6])
    X7 = Multi_Path_Inception_Block()(sum4)
    sum5 = add(inputs=[sum4,X7])
    X8 = Multi_Path_Inception_Block()(sum5)
    sum6 = add(inputs=[sum5,X8])
    X9 = Multi_Path_Inception_Block()(sum6)
    sum7 = add(inputs=[sum6,X9])
    X10 = Multi_Path_Inception_Block()(sum7)
    sum8 = add(inputs=[sum7,X10])
    XA = Conv2D(32, (3,3), padding='same', activation='relu')(sum8)

    Y = Conv2DTranspose(filters=3, kernel_size = (9,9), strides = (scale,scale), padding="same",activation=None)(XA)

    X_out = depth_to_space(Y, scale)
    model = Model(inputs=X_in, outputs=X_out)


    model.compile(optimizer=Adam(learning_rate=1e-3),loss=MeanSquaredError(),metrics=[hf.PSNR])

    return model

In [7]:
from tabnanny import verbose
import tensorflow as tf
from Model import IRMRIS
import pandas as pd
import HelpFunc as hf
import numpy as np
import argparse
import os

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument('--epoch', type=int, default=100, help='-')
parser.add_argument('--batch_size', type=int, default=128, help='-')

FLAGS, unparsed = parser.parse_known_args()

X_tr = np.load("dataset/data_train.npy")
Y_tr = np.load("dataset/labels_train.npy")

X_val = np.load("dataset/data_validation.npy")
Y_val = np.load("dataset/labels_validation.npy")

X_te = np.load("dataset/data_test.npy")
Y_te = np.load("dataset/labels_test.npy")

model = IRMRIS()

ckpt_path = "checkpoint/model-ckpt"
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=ckpt_path,
    save_weights_only=True,
    verbose=1
)

if os.path.exists(f"{ckpt_path}.index"):
    print(f"\n\nLoad Model from \"{ckpt_path}\"")
    model.load_weights(ckpt_path)
else:
    print(f"\n\n\"{ckpt_path}\" DO NOT EXIST")


if os.path.exists("checkpoint/cur_epoch.npy"):
    cur_epoch = np.load("checkpoint/cur_epoch.npy")
else:
    cur_epoch = np.int32(0)
print(f"\n\nCurrent epoch is {cur_epoch}\n")


print("\n\n==================================================",
      " Train ",
      "==================================================\n")

history = model.fit(
    x=X_tr, y=Y_tr,
    batch_size=FLAGS.batch_size,
    epochs=FLAGS.epoch,
    validation_data=(X_val, Y_val),
    callbacks=[ckpt_callback],
    verbose=1
)

print("\n\n==================================================",
      " Evaluate ",
      "==================================================\n")

model.evaluate(X_te, Y_te)

df = pd.DataFrame(history.history)
df = hf.AppendHistory(df)
df.to_csv("history.csv", index=False)

cur_epoch = cur_epoch + FLAGS.epoch
print(f"\nTotal trained epochs is {cur_epoch}\n")

np.save("checkpoint/cur_epoch.npy", cur_epoch)



"checkpoint/model-ckpt" DO NOT EXIST


Current epoch is 0



==================================================  Train  ==================================================

Epoch 1/100
394/394 [==============================] - ETA: 0s - loss: 0.0655 - PSNR: 13.3196
Epoch 00001: saving model to checkpoint\model-ckpt
394/394 [==============================] - 6s 7ms/step - loss: 0.0655 - PSNR: 13.3196 - val_loss: 0.0113 - val_PSNR: 19.4665
Epoch 2/100
393/394 [============================>.] - ETA: 0s - loss: 0.0084 - PSNR: 20.8010
Epoch 00002: saving model to checkpoint\model-ckpt
394/394 [==============================] - 2s 6ms/step - loss: 0.0084 - PSNR: 20.8028 - val_loss: 0.0070 - val_PSNR: 21.5523
Epoch 3/100
387/394 [============================>.] - ETA: 0s - loss: 0.0062 - PSNR: 22.0802
Epoch 00003: saving model to checkpoint\model-ckpt
394/394 [==============================] - 2s 6ms/step - loss: 0.0062 - PSNR: 22.0877 - val_loss: 0.0059 - val_PSNR: 22.3314
Epoch 4/100
394/

Epoch 31/100
391/394 [============================>.] - ETA: 0s - loss: 0.0019 - PSNR: 27.3375
Epoch 00031: saving model to checkpoint\model-ckpt
394/394 [==============================] - 3s 8ms/step - loss: 0.0019 - PSNR: 27.3358 - val_loss: 0.0020 - val_PSNR: 26.9454
Epoch 32/100
394/394 [==============================] - ETA: 0s - loss: 0.0019 - PSNR: 27.3098
Epoch 00032: saving model to checkpoint\model-ckpt
394/394 [==============================] - 3s 8ms/step - loss: 0.0019 - PSNR: 27.3098 - val_loss: 0.0020 - val_PSNR: 26.9738
Epoch 33/100
390/394 [============================>.] - ETA: 0s - loss: 0.0019 - PSNR: 27.3609
Epoch 00033: saving model to checkpoint\model-ckpt
394/394 [==============================] - 3s 8ms/step - loss: 0.0019 - PSNR: 27.3615 - val_loss: 0.0020 - val_PSNR: 26.9756
Epoch 34/100
388/394 [============================>.] - ETA: 0s - loss: 0.0019 - PSNR: 27.3422
Epoch 00034: saving model to checkpoint\model-ckpt
394/394 [==============================] 

393/394 [============================>.] - ETA: 0s - loss: 0.0018 - PSNR: 27.3707
Epoch 00061: saving model to checkpoint\model-ckpt
394/394 [==============================] - 3s 6ms/step - loss: 0.0018 - PSNR: 27.3702 - val_loss: 0.0020 - val_PSNR: 26.9872
Epoch 62/100
391/394 [============================>.] - ETA: 0s - loss: 0.0018 - PSNR: 27.3691
Epoch 00062: saving model to checkpoint\model-ckpt
394/394 [==============================] - 3s 7ms/step - loss: 0.0018 - PSNR: 27.3701 - val_loss: 0.0020 - val_PSNR: 26.9896
Epoch 63/100
387/394 [============================>.] - ETA: 0s - loss: 0.0018 - PSNR: 27.3686
Epoch 00063: saving model to checkpoint\model-ckpt
394/394 [==============================] - 3s 7ms/step - loss: 0.0018 - PSNR: 27.3705 - val_loss: 0.0020 - val_PSNR: 26.9551
Epoch 64/100
386/394 [============================>.] - ETA: 0s - loss: 0.0018 - PSNR: 27.3724
Epoch 00064: saving model to checkpoint\model-ckpt
394/394 [==============================] - 3s 7ms/step

389/394 [============================>.] - ETA: 0s - loss: 0.0018 - PSNR: 27.3824
Epoch 00091: saving model to checkpoint\model-ckpt
394/394 [==============================] - 2s 6ms/step - loss: 0.0018 - PSNR: 27.3762 - val_loss: 0.0020 - val_PSNR: 26.9887
Epoch 92/100
394/394 [==============================] - ETA: 0s - loss: 0.0018 - PSNR: 27.3750
Epoch 00092: saving model to checkpoint\model-ckpt
394/394 [==============================] - 2s 6ms/step - loss: 0.0018 - PSNR: 27.3750 - val_loss: 0.0020 - val_PSNR: 26.9916
Epoch 93/100
393/394 [============================>.] - ETA: 0s - loss: 0.0018 - PSNR: 27.3838
Epoch 00093: saving model to checkpoint\model-ckpt
394/394 [==============================] - 3s 6ms/step - loss: 0.0018 - PSNR: 27.3832 - val_loss: 0.0020 - val_PSNR: 26.9934
Epoch 94/100
394/394 [==============================] - ETA: 0s - loss: 0.0018 - PSNR: 27.3799
Epoch 00094: saving model to checkpoint\model-ckpt
394/394 [==============================] - 3s 6ms/step